En este codigo se tiene que cargar el vectorizer y después se puede ejecutar cualquier grupo de celdas para generar los archivos correspondientes.

# 0. Cargamos vectorizer

In [27]:
from google.colab import drive

In [28]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import pandas as pd

In [30]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
stop_words = set(stopwords.words("spanish"))
lemmatizer = WordNetLemmatizer()

In [33]:
texto1 = 'Hola soy Luciano'
texto2 = 'Hola soy Luciano y me gusta jugar futbol'

In [34]:
text1_tokenized = ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(texto1) if word.isalnum() and word.lower() not in stop_words
    ])
text2_tokenized = ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(texto2) if word.isalnum() and word.lower() not in stop_words
    ])

In [35]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([text1_tokenized, text2_tokenized])

In [36]:
similitud = 1 - (tfidf_matrix * tfidf_matrix.T).toarray()[0, 1]
similitud

0.4976712217743283


# 1. Dataset completo


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [ ]:
descriptions = df.tokens.tolist()

In [ ]:
def tokenize(description, lemmatizer, stop_words) -> str:
    return ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(description) if word.isalnum() and word.lower() not in stop_words
    ])

def tokenize_descriptions(descriptions, lemmatizer, stop_words) -> list:
    tokenized = [tokenize(desc, lemmatizer, stop_words) for desc in descriptions]
    return tokenized

In [ ]:
tokenized_descriptions = tokenize_descriptions(descriptions, lemmatizer, stop_words)

In [ ]:
def get_vectors_tf_idf(descriptions, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    return tfidf_matrix.toarray()

In [ ]:
description_vectors_tfidf = get_vectors_tf_idf(tokenized_descriptions, vectorizer).tolist()

In [ ]:
df['tfidf_vector'] = description_vectors_tfidf

In [ ]:
import numpy as np
import time
from numpy.linalg import norm

In [ ]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [ ]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.tfidf_vector[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [ ]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         14100
Chimu          4791
Wari           4712
Nasca          2994
Sican          1333
Chancay         916
Cajamarca       882
Tiahuanaco      699
Salinar         669
Cupisnique      522
Vicus           363
Recuay          327
Name: culture_cl, dtype: int64

In [ ]:
df.to_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_tfidf.csv', sep=';', index=False)

In [ ]:
seconds_per_data = 0.000110168477982
distances = {p: list() for p in culture_pairs}

for i in range(0,len(culture_pairs)):
    start_time = time.time()
    c1 = culture_pairs[i][0]
    c2 = culture_pairs[i][1]

    print(f'{i} -> Cultures: {c1}, {c2}')
    df_c1 = df[df.culture_cl == c1]
    df_c2 = df[df.culture_cl == c2]

    # Time estimation
    len_c1 = len(df_c1)
    len_c2 = len(df_c2)
    total_data = len_c1 * len_c2
    estimated_s = total_data * seconds_per_data
    estimated_minute = '0' + str(round(estimated_s / 60)) if round(estimated_s / 60) < 10 else str(round(estimated_s / 60))
    estimated_second = '0' + str(np.round(estimated_s % 60, 2)) if np.round(estimated_s % 60, 2) < 10 else str(np.round(estimated_s % 60, 2))
    print(f'Estimated time in seconds: {np.round(estimated_s, 2)}')
    print(f'Estimated time in minutes: {estimated_minute}:{estimated_second}')

    embed_c1 = df_c1.tfidf_vector
    embed_c2 = df_c2.tfidf_vector

    for e1 in embed_c1:
        for e2 in embed_c2:
            dist = 1 - _cosine_similarity(e1, e2)
            distances[(c1, c2)].append(dist)

    end_time = time.time()

    # Save distances in file
    mean = np.mean(distances[(c1, c2)])
    file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_tfidf_dataset_completo.txt', 'a')
    file.write(f'{i}: {c1}-{c2} -> {mean}\n')
    file.close()

    print(f'Time distances between: {end_time - start_time}s')
    print(f'Mean distance: {mean}')
    print()

34 -> Cultures: Wari, Chimu
Estimated time in seconds: 2487.07
Estimated time in minutes: 41:27.07
Time distances between: 11261.12075138092s
Mean distance: 0.912454543839789

35 -> Cultures: Wari, Chancay
Estimated time in seconds: 475.51
Estimated time in minutes: 08:55.51
Time distances between: 1750.2634320259094s
Mean distance: 0.908816004059549

36 -> Cultures: Wari, Salinar
Estimated time in seconds: 347.29
Estimated time in minutes: 06:47.29
Time distances between: 1281.0285799503326s
Mean distance: 0.8835088187172093

37 -> Cultures: Wari, Tiahuanaco
Estimated time in seconds: 362.86
Estimated time in minutes: 06:02.86
Time distances between: 1290.699224948883s
Mean distance: 0.9055076456334527

38 -> Cultures: Cupisnique, Cajamarca
Estimated time in seconds: 50.72
Estimated time in minutes: 01:50.72
Time distances between: 121.98314166069031s
Mean distance: 0.9159263407459818

39 -> Cultures: Cupisnique, Vicus
Estimated time in seconds: 20.88
Estimated time in minutes: 00:20.

# 1.1 Por periodo

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

In [ ]:
descriptions = df.tokens.tolist()

In [ ]:
def tokenize(description, lemmatizer, stop_words) -> str:
    return ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(description) if word.isalnum() and word.lower() not in stop_words
    ])

def tokenize_descriptions(descriptions, lemmatizer, stop_words) -> list:
    tokenized = [tokenize(desc, lemmatizer, stop_words) for desc in descriptions]
    return tokenized

In [ ]:
tokenized_descriptions = tokenize_descriptions(descriptions, lemmatizer, stop_words)

In [ ]:
def get_vectors_tf_idf(descriptions, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    return tfidf_matrix.toarray()

In [ ]:
description_vectors_tfidf = get_vectors_tf_idf(tokenized_descriptions, vectorizer).tolist()

In [ ]:
df['tfidf_vector'] = description_vectors_tfidf

In [ ]:
import numpy as np
import time
from numpy.linalg import norm

In [ ]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [ ]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.tfidf_vector[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [ ]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         14100
Chimu          4791
Wari           4712
Nasca          2994
Sican          1333
Chancay         916
Cajamarca       882
Tiahuanaco      699
Salinar         669
Cupisnique      522
Vicus           363
Recuay          327
Name: culture_cl, dtype: int64

In [ ]:
ctimes = df.cronology_time.unique().tolist()
ctimes

[7, 6, 5, 8]

In [ ]:
for ctime in ctimes:
    filtered_df = df[df['cronology_time'] == ctime]
    print(f'Cronology_time: {ctime}')
    display(filtered_df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False))

    cultures = filtered_df.culture_cl.unique().tolist()
    culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
    print(f'Culture pairs: {len(culture_pairs)}')

    distances = {p: list() for p in culture_pairs}

    for i in range(0, len(culture_pairs)):
        start_time = time.time()
        c1 = culture_pairs[i][0]
        c2 = culture_pairs[i][1]

        print(f'{i} -> Cultures: {c1}, {c2}')
        df_c1 = filtered_df[filtered_df.culture_cl == c1]
        df_c2 = filtered_df[filtered_df.culture_cl == c2]

        embed_c1 = df_c1.tfidf_vector
        embed_c2 = df_c2.tfidf_vector

        for e1 in embed_c1:
            for e2 in embed_c2:
                dist = 1 - _cosine_similarity(e1, e2)
                distances[(c1, c2)].append(dist)

        end_time = time.time()

        # Save distances in file
        mean = np.mean(distances[(c1, c2)])
        file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_tfidf_dataset_completo_ctime_{ctime}.txt', 'a')
        file.write(f'{i}: {c1}-{c2} -> {mean}\n')
        file.close()

        print(f'Time distances between: {end_time - start_time}s')
        print(f'Mean distance: {mean}')
        print()

Cronology_time: 7


culture_cl
Wari          4712
Sican         1333
Tiahuanaco     699
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Sican, Wari
Time distances between: 2766.5236876010895s
Mean distance: 0.9183981027969552

1 -> Cultures: Sican, Tiahuanaco
Time distances between: 295.8649437427521s
Mean distance: 0.9539904244156333

2 -> Cultures: Wari, Tiahuanaco
Time distances between: 1292.8560690879822s
Mean distance: 0.9055076456334527

Cronology_time: 6


culture_cl
Moche        14100
Nasca         2994
Cajamarca      882
Recuay         327
Name: culture_cl, dtype: int64

Culture pairs: 6
0 -> Cultures: Recuay, Moche
Time distances between: 2103.75191283226s
Mean distance: 0.9344248061605729

1 -> Cultures: Recuay, Cajamarca
Time distances between: 74.85828351974487s
Mean distance: 0.9622780972250345

2 -> Cultures: Recuay, Nasca
Time distances between: 417.92284655570984s
Mean distance: 0.942403669463586

3 -> Cultures: Moche, Cajamarca
Time distances between: 3002.216945409775s
Mean distance: 0.9771703363967726

4 -> Cultures: Moche, Nasca
Time distances between: 18906.83812904358s
Mean distance: 0.9567462053288333

5 -> Cultures: Cajamarca, Nasca
Time distances between: 859.5776739120483s
Mean distance: 0.9163425656247751

Cronology_time: 5


culture_cl
Salinar       669
Cupisnique    522
Vicus         363
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Cupisnique, Vicus
Time distances between: 81.19182586669922s
Mean distance: 0.9107011552486992

1 -> Cultures: Cupisnique, Salinar
Time distances between: 127.57623887062073s
Mean distance: 0.8999103158918729

2 -> Cultures: Vicus, Salinar
Time distances between: 113.51730179786682s
Mean distance: 0.8598509846174717

Cronology_time: 8


culture_cl
Chimu      4791
Chancay     916
Name: culture_cl, dtype: int64

Culture pairs: 1
0 -> Cultures: Chimu, Chancay
Time distances between: 1438.0086743831635s
Mean distance: 0.9512874056659766



#2. Dataset Menos 1

In [37]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [38]:
df = df.loc[df['numero_palabras'] != 1]

In [39]:
descriptions = df.tokens.tolist()

In [40]:
def tokenize(description, lemmatizer, stop_words) -> str:
    return ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(description) if word.isalnum() and word.lower() not in stop_words
    ])

def tokenize_descriptions(descriptions, lemmatizer, stop_words) -> list:
    tokenized = [tokenize(desc, lemmatizer, stop_words) for desc in descriptions]
    return tokenized

In [41]:
tokenized_descriptions = tokenize_descriptions(descriptions, lemmatizer, stop_words)

In [42]:
def get_vectors_tf_idf(descriptions, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    return tfidf_matrix.toarray()

In [43]:
description_vectors_tfidf = get_vectors_tf_idf(tokenized_descriptions, vectorizer).tolist()

In [44]:
df['tfidf_vector'] = description_vectors_tfidf

<ipython-input-44-a7057baffbea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tfidf_vector'] = description_vectors_tfidf


In [45]:
import numpy as np
import time
from numpy.linalg import norm

In [46]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [47]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.tfidf_vector[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [48]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         13637
Chimu          4594
Wari           4241
Nasca          2963
Sican          1331
Chancay         697
Cajamarca       686
Tiahuanaco      653
Salinar         587
Cupisnique      358
Vicus           355
Recuay          308
Name: culture_cl, dtype: int64

In [49]:
seconds_per_data = 0.000110168477982
distances = {p: list() for p in culture_pairs}

for i in range(0,len(culture_pairs)):
    start_time = time.time()
    c1 = culture_pairs[i][0]
    c2 = culture_pairs[i][1]

    print(f'{i} -> Cultures: {c1}, {c2}')
    df_c1 = df[df.culture_cl == c1]
    df_c2 = df[df.culture_cl == c2]

    # Time estimation
    len_c1 = len(df_c1)
    len_c2 = len(df_c2)
    total_data = len_c1 * len_c2
    estimated_s = total_data * seconds_per_data
    estimated_minute = '0' + str(round(estimated_s / 60)) if round(estimated_s / 60) < 10 else str(round(estimated_s / 60))
    estimated_second = '0' + str(np.round(estimated_s % 60, 2)) if np.round(estimated_s % 60, 2) < 10 else str(np.round(estimated_s % 60, 2))
    print(f'Estimated time in seconds: {np.round(estimated_s, 2)}')
    print(f'Estimated time in minutes: {estimated_minute}:{estimated_second}')

    embed_c1 = df_c1.tfidf_vector
    embed_c2 = df_c2.tfidf_vector

    for e1 in embed_c1:
        for e2 in embed_c2:
            dist = 1 - _cosine_similarity(e1, e2)
            distances[(c1, c2)].append(dist)

    end_time = time.time()

    # Save distances in file
    mean = np.mean(distances[(c1, c2)])
    file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_tfidf_dataset_menos_uno.txt', 'a')
    file.write(f'{i}: {c1}-{c2} -> {mean}\n')
    file.close()

    print(f'Time distances between: {end_time - start_time}s')
    print(f'Mean distance: {mean}')
    print()

26 -> Cultures: Moche, Chimu
Estimated time in seconds: 6901.88
Estimated time in minutes: 115:01.88


KeyboardInterrupt: 

# 2.1 Por periodo

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [ ]:
df = df.loc[df['numero_palabras'] != 1]

In [ ]:
descriptions = df.tokens.tolist()

In [ ]:
def tokenize(description, lemmatizer, stop_words) -> str:
    return ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(description) if word.isalnum() and word.lower() not in stop_words
    ])

def tokenize_descriptions(descriptions, lemmatizer, stop_words) -> list:
    tokenized = [tokenize(desc, lemmatizer, stop_words) for desc in descriptions]
    return tokenized

In [ ]:
tokenized_descriptions = tokenize_descriptions(descriptions, lemmatizer, stop_words)

In [ ]:
def get_vectors_tf_idf(descriptions, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    return tfidf_matrix.toarray()

In [ ]:
description_vectors_tfidf = get_vectors_tf_idf(tokenized_descriptions, vectorizer).tolist()

In [ ]:
df['tfidf_vector'] = description_vectors_tfidf

<ipython-input-33-a7057baffbea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tfidf_vector'] = description_vectors_tfidf


In [ ]:
import numpy as np
import time
from numpy.linalg import norm

In [ ]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [ ]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.tfidf_vector[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [ ]:
ctimes = df.cronology_time.unique().tolist()
ctimes

[7, 6, 5, 8]

In [ ]:
for ctime in ctimes:
    filtered_df = df[df['cronology_time'] == ctime]
    print(f'Cronology_time: {ctime}')
    display(filtered_df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False))

    cultures = filtered_df.culture_cl.unique().tolist()
    culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
    print(f'Culture pairs: {len(culture_pairs)}')

    distances = {p: list() for p in culture_pairs}

    for i in range(0, len(culture_pairs)):
        start_time = time.time()
        c1 = culture_pairs[i][0]
        c2 = culture_pairs[i][1]

        print(f'{i} -> Cultures: {c1}, {c2}')
        df_c1 = filtered_df[filtered_df.culture_cl == c1]
        df_c2 = filtered_df[filtered_df.culture_cl == c2]

        embed_c1 = df_c1.tfidf_vector
        embed_c2 = df_c2.tfidf_vector

        for e1 in embed_c1:
            for e2 in embed_c2:
                dist = 1 - _cosine_similarity(e1, e2)
                distances[(c1, c2)].append(dist)

        end_time = time.time()

        # Save distances in file
        mean = np.mean(distances[(c1, c2)])
        file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_tfidf_dataset_menos_uno_ctime_{ctime}.txt', 'a')
        file.write(f'{i}: {c1}-{c2} -> {mean}\n')
        file.close()

        print(f'Time distances between: {end_time - start_time}s')
        print(f'Mean distance: {mean}')
        print()

Cronology_time: 8


culture_cl
Chimu      4594
Chancay     697
Name: culture_cl, dtype: int64

Culture pairs: 1
0 -> Cultures: Chimu, Chancay
Time distances between: 1370.5986471176147s
Mean distance: 0.9458980576740427



# 3. Dataset Menos 2

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

In [ ]:
df = df.loc[df['numero_palabras'] != 1]
df = df.loc[df['numero_palabras'] != 2]

In [ ]:
def tokenize(description, lemmatizer, stop_words) -> str:
    return ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(description) if word.isalnum() and word.lower() not in stop_words
    ])

def tokenize_descriptions(descriptions, lemmatizer, stop_words) -> list:
    tokenized = [tokenize(desc, lemmatizer, stop_words) for desc in descriptions]
    return tokenized

In [ ]:
tokenized_descriptions = tokenize_descriptions(descriptions, lemmatizer, stop_words)

In [ ]:
def get_vectors_tf_idf(descriptions, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    return tfidf_matrix.toarray()

In [ ]:
description_vectors_tfidf = get_vectors_tf_idf(tokenized_descriptions, vectorizer).tolist()

In [ ]:
df['tfidf_vector'] = description_vectors_tfidf

In [ ]:
import numpy as np
import time
from numpy.linalg import norm

In [ ]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

In [ ]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.tfidf_vector[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [ ]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

In [ ]:
seconds_per_data = 0.000110168477982
distances = {p: list() for p in culture_pairs}

for i in range(0,len(culture_pairs)):
    start_time = time.time()
    c1 = culture_pairs[i][0]
    c2 = culture_pairs[i][1]

    print(f'{i} -> Cultures: {c1}, {c2}')
    df_c1 = df[df.culture_cl == c1]
    df_c2 = df[df.culture_cl == c2]

    # Time estimation
    len_c1 = len(df_c1)
    len_c2 = len(df_c2)
    total_data = len_c1 * len_c2
    estimated_s = total_data * seconds_per_data
    estimated_minute = '0' + str(round(estimated_s / 60)) if round(estimated_s / 60) < 10 else str(round(estimated_s / 60))
    estimated_second = '0' + str(np.round(estimated_s % 60, 2)) if np.round(estimated_s % 60, 2) < 10 else str(np.round(estimated_s % 60, 2))
    print(f'Estimated time in seconds: {np.round(estimated_s, 2)}')
    print(f'Estimated time in minutes: {estimated_minute}:{estimated_second}')

    embed_c1 = df_c1.tfidf_vector
    embed_c2 = df_c2.tfidf_vector

    for e1 in embed_c1:
        for e2 in embed_c2:
            dist = 1 - _cosine_similarity(e1, e2)
            distances[(c1, c2)].append(dist)

    end_time = time.time()

    # Save distances in file
    mean = np.mean(distances[(c1, c2)])
    file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_tfidf_dataset_menos_dos.txt', 'a')
    file.write(f'{i}: {c1}-{c2} -> {mean}\n')
    file.close()

    print(f'Time distances between: {end_time - start_time}s')
    print(f'Mean distance: {mean}')
    print()

# 3.1 Por periodo

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [ ]:
df = df.loc[df['numero_palabras'] != 1]
df = df.loc[df['numero_palabras'] != 2]

In [ ]:
descriptions = df.tokens.tolist()

In [ ]:
def tokenize(description, lemmatizer, stop_words) -> str:
    return ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(description) if word.isalnum() and word.lower() not in stop_words
    ])

def tokenize_descriptions(descriptions, lemmatizer, stop_words) -> list:
    tokenized = [tokenize(desc, lemmatizer, stop_words) for desc in descriptions]
    return tokenized

In [ ]:
tokenized_descriptions = tokenize_descriptions(descriptions, lemmatizer, stop_words)

In [ ]:
def get_vectors_tf_idf(descriptions, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    return tfidf_matrix.toarray()

In [ ]:
description_vectors_tfidf = get_vectors_tf_idf(tokenized_descriptions, vectorizer).tolist()

In [ ]:
df['tfidf_vector'] = description_vectors_tfidf

In [ ]:
import numpy as np
import time
from numpy.linalg import norm

In [ ]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [ ]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.tfidf_vector[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [ ]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         13415
Chimu          4546
Wari           4145
Nasca          2959
Sican          1331
Chancay         685
Tiahuanaco      653
Salinar         582
Cajamarca       520
Vicus           355
Cupisnique      354
Recuay          302
Name: culture_cl, dtype: int64

In [ ]:
ctimes = df.cronology_time.unique().tolist()
ctimes

[7, 6, 5, 8]

In [ ]:
for ctime in ctimes:
    filtered_df = df[df['cronology_time'] == ctime]
    print(f'Cronology_time: {ctime}')
    display(filtered_df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False))

    cultures = filtered_df.culture_cl.unique().tolist()
    culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
    print(f'Culture pairs: {len(culture_pairs)}')

    distances = {p: list() for p in culture_pairs}

    for i in range(0, len(culture_pairs)):
        start_time = time.time()
        c1 = culture_pairs[i][0]
        c2 = culture_pairs[i][1]

        print(f'{i} -> Cultures: {c1}, {c2}')
        df_c1 = filtered_df[filtered_df.culture_cl == c1]
        df_c2 = filtered_df[filtered_df.culture_cl == c2]

        embed_c1 = df_c1.tfidf_vector
        embed_c2 = df_c2.tfidf_vector

        for e1 in embed_c1:
            for e2 in embed_c2:
                dist = 1 - _cosine_similarity(e1, e2)
                distances[(c1, c2)].append(dist)

        end_time = time.time()

        # Save distances in file
        mean = np.mean(distances[(c1, c2)])
        file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_tfidf_dataset_menos_dos_ctime_{ctime}.txt', 'a')
        file.write(f'{i}: {c1}-{c2} -> {mean}\n')
        file.close()

        print(f'Time distances between: {end_time - start_time}s')
        print(f'Mean distance: {mean}')
        print()

Cronology_time: 7


culture_cl
Wari          4145
Sican         1331
Tiahuanaco     653
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Sican, Wari
Time distances between: 2921.413995742798s
Mean distance: 0.9112057866507629

1 -> Cultures: Sican, Tiahuanaco
Time distances between: 313.9912588596344s
Mean distance: 0.9527586098849519

2 -> Cultures: Wari, Tiahuanaco
Time distances between: 1279.7448499202728s
Mean distance: 0.8990840096725318

Cronology_time: 6


culture_cl
Moche        13415
Nasca         2959
Cajamarca      520
Recuay         302
Name: culture_cl, dtype: int64

Culture pairs: 6
0 -> Cultures: Recuay, Moche
Time distances between: 2222.628522634506s
Mean distance: 0.9300243563431637

1 -> Cultures: Recuay, Cajamarca
Time distances between: 59.614476680755615s
Mean distance: 0.9402029874014761

2 -> Cultures: Recuay, Nasca
Time distances between: 449.9242615699768s
Mean distance: 0.939771135671469

3 -> Cultures: Moche, Cajamarca
Time distances between: 2387.309994697571s
Mean distance: 0.9619535378626154

4 -> Cultures: Moche, Nasca
Time distances between: 19197.42117190361s
Mean distance: 0.9563432595796966

5 -> Cultures: Cajamarca, Nasca
Time distances between: 588.3667957782745s
Mean distance: 0.9146821614632542

Cronology_time: 5


culture_cl
Salinar       582
Vicus         355
Cupisnique    354
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Cupisnique, Vicus
Time distances between: 71.19940280914307s
Mean distance: 0.8801131846316739

1 -> Cultures: Cupisnique, Salinar
Time distances between: 103.32270693778992s
Mean distance: 0.8608923979224636

2 -> Cultures: Vicus, Salinar
Time distances between: 109.30455493927002s
Mean distance: 0.8444056007768652

Cronology_time: 8


culture_cl
Chimu      4546
Chancay     685
Name: culture_cl, dtype: int64

Culture pairs: 1
0 -> Cultures: Chimu, Chancay
Time distances between: 1236.4318537712097s
Mean distance: 0.9457814871184572



# 4. Dataset Menos 3

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [ ]:
df = df.loc[df['numero_palabras'] != 1]
df = df.loc[df['numero_palabras'] != 2]
df = df.loc[df['numero_palabras'] != 3]

In [ ]:
def tokenize(description, lemmatizer, stop_words) -> str:
    return ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(description) if word.isalnum() and word.lower() not in stop_words
    ])

def tokenize_descriptions(descriptions, lemmatizer, stop_words) -> list:
    tokenized = [tokenize(desc, lemmatizer, stop_words) for desc in descriptions]
    return tokenized

In [ ]:
tokenized_descriptions = tokenize_descriptions(descriptions, lemmatizer, stop_words)

KeyboardInterrupt: 

In [ ]:
def get_vectors_tf_idf(descriptions, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    return tfidf_matrix.toarray()

In [ ]:
description_vectors_tfidf = get_vectors_tf_idf(tokenized_descriptions, vectorizer).tolist()

In [ ]:
df['tfidf_vector'] = description_vectors_tfidf

In [ ]:
import numpy as np
import time
from numpy.linalg import norm

In [ ]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

In [ ]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.tfidf_vector[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [ ]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

In [ ]:
seconds_per_data = 0.000110168477982
distances = {p: list() for p in culture_pairs}

for i in range(0,len(culture_pairs)):
    start_time = time.time()
    c1 = culture_pairs[i][0]
    c2 = culture_pairs[i][1]

    print(f'{i} -> Cultures: {c1}, {c2}')
    df_c1 = df[df.culture_cl == c1]
    df_c2 = df[df.culture_cl == c2]

    # Time estimation
    len_c1 = len(df_c1)
    len_c2 = len(df_c2)
    total_data = len_c1 * len_c2
    estimated_s = total_data * seconds_per_data
    estimated_minute = '0' + str(round(estimated_s / 60)) if round(estimated_s / 60) < 10 else str(round(estimated_s / 60))
    estimated_second = '0' + str(np.round(estimated_s % 60, 2)) if np.round(estimated_s % 60, 2) < 10 else str(np.round(estimated_s % 60, 2))
    print(f'Estimated time in seconds: {np.round(estimated_s, 2)}')
    print(f'Estimated time in minutes: {estimated_minute}:{estimated_second}')

    embed_c1 = df_c1.tfidf_vector
    embed_c2 = df_c2.tfidf_vector

    for e1 in embed_c1:
        for e2 in embed_c2:
            dist = 1 - _cosine_similarity(e1, e2)
            distances[(c1, c2)].append(dist)

    end_time = time.time()

    # Save distances in file
    mean = np.mean(distances[(c1, c2)])
    file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_tfidf_dataset_menos_tres.txt', 'a')
    file.write(f'{i}: {c1}-{c2} -> {mean}\n')
    file.close()

    print(f'Time distances between: {end_time - start_time}s')
    print(f'Mean distance: {mean}')
    print()

# 4.1 Por periodo

In [12]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150.csv',  sep=';')
df

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n101,trait_n102,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,botella doble cuerpo asa puente cintada silbad...,21
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,botella doble pico asa puente cintada escultor...,29
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,botella gollete asa puente cintada protoma sil...,16
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,botella gollete asa puente cintada protoma sil...,15
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,1,1,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,botella gollete asa puente cintada protoma sil...,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32303,33495,ML038664,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de escalonados y...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038664a.jpg,58,plato disenos geometricos escalonados triangulos,5
32304,33496,ML038665,Horizonte Medio,7,Tiahuanaco,plato,"plato con disenos geometricos de semicirculo, ...",NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038665a.jpg,67,plato disenos geometricos semicirculo escalona...,6
32305,33497,ML038666,Horizonte Medio,7,Tiahuanaco,plato,plato con representar de cabeza de personajes ...,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038666a.jpg,56,plato representar cabeza personajes zoomorfos,5
32306,33498,ML038667,Horizonte Medio,7,Tiahuanaco,plato,plato con disenos geometricos de semicirculo.,NaN,pintado,naranja,...,1,1,0,1,0,data/tiahuanaco_7/16 ADMINISTRADOR COLECCIONES...,data/images/ML038667a.jpg,45,plato disenos geometricos semicirculo,4


In [13]:
df = df.loc[df['numero_palabras'] != 1]
df = df.loc[df['numero_palabras'] != 2]
df = df.loc[df['numero_palabras'] != 3]

In [14]:
descriptions = df.tokens.tolist()

In [15]:
def tokenize(description, lemmatizer, stop_words) -> str:
    return ' '.join([
    lemmatizer.lemmatize(word.lower())
        for word in word_tokenize(description) if word.isalnum() and word.lower() not in stop_words
    ])

def tokenize_descriptions(descriptions, lemmatizer, stop_words) -> list:
    tokenized = [tokenize(desc, lemmatizer, stop_words) for desc in descriptions]
    return tokenized

In [16]:
tokenized_descriptions = tokenize_descriptions(descriptions, lemmatizer, stop_words)

In [17]:
def get_vectors_tf_idf(descriptions, vectorizer):
    tfidf_matrix = vectorizer.fit_transform(descriptions)
    return tfidf_matrix.toarray()

In [18]:
description_vectors_tfidf = get_vectors_tf_idf(tokenized_descriptions, vectorizer).tolist()

In [19]:
df['tfidf_vector'] = description_vectors_tfidf

In [20]:
import numpy as np
import time
from numpy.linalg import norm

In [21]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [22]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.tfidf_vector[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [23]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         13207
Chimu          4497
Wari           4025
Nasca          2428
Sican          1329
Chancay         661
Tiahuanaco      616
Salinar         576
Cajamarca       514
Vicus           355
Cupisnique      340
Recuay          296
Name: culture_cl, dtype: int64

In [24]:
ctimes = df.cronology_time.unique().tolist()
ctimes

[7, 6, 5, 8]

In [26]:
for ctime in ctimes:
    filtered_df = df[df['cronology_time'] == ctime]
    print(f'Cronology_time: {ctime}')
    display(filtered_df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False))

    cultures = filtered_df.culture_cl.unique().tolist()
    culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
    print(f'Culture pairs: {len(culture_pairs)}')

    distances = {p: list() for p in culture_pairs}

    for i in range(0, len(culture_pairs)):
        start_time = time.time()
        c1 = culture_pairs[i][0]
        c2 = culture_pairs[i][1]

        print(f'{i} -> Cultures: {c1}, {c2}')
        df_c1 = filtered_df[filtered_df.culture_cl == c1]
        df_c2 = filtered_df[filtered_df.culture_cl == c2]

        embed_c1 = df_c1.tfidf_vector
        embed_c2 = df_c2.tfidf_vector

        for e1 in embed_c1:
            for e2 in embed_c2:
                dist = 1 - _cosine_similarity(e1, e2)
                distances[(c1, c2)].append(dist)

        end_time = time.time()

        # Save distances in file
        mean = np.mean(distances[(c1, c2)])
        file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_tfidf_dataset_menos_tres_ctime_{ctime}.txt', 'a')
        file.write(f'{i}: {c1}-{c2} -> {mean}\n')
        file.close()

        print(f'Time distances between: {end_time - start_time}s')
        print(f'Mean distance: {mean}')
        print()

Cronology_time: 5


culture_cl
Salinar       576
Vicus         355
Cupisnique    340
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Cupisnique, Vicus
Time distances between: 75.84084224700928s
Mean distance: 0.878890384093443

1 -> Cultures: Cupisnique, Salinar
Time distances between: 109.37416362762451s
Mean distance: 0.8585030926794259

2 -> Cultures: Vicus, Salinar
Time distances between: 119.11687731742859s
Mean distance: 0.8449991908985967

Cronology_time: 8


culture_cl
Chimu      4497
Chancay     661
Name: culture_cl, dtype: int64

Culture pairs: 1
0 -> Cultures: Chimu, Chancay
Time distances between: 1268.5940613746643s
Mean distance: 0.9462987836305976

